<a href="https://colab.research.google.com/github/ariel-hedvat/AdvancedMLDLCourseAssignments/blob/main/Experiments/data_extractuon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [96]:
#installation for spotipy-API
!pip install spotipy

In [97]:
#imports
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials
import os
import zipfile
import pandas as pd
import requests
import io
import subprocess

In [98]:
# Function to read Spotify credentials from a URL
def read_spotify_credentials_from_url(url):
    credentials = {}
    response = requests.get(url)
    if response.status_code == 200:
        lines = response.text.split('\n')
        for line in lines:
            if '=' in line:
                key, value = line.strip().split('=')
                credentials[key] = value
    else:
        print("Failed to fetch Spotify credentials. Status code:", response.status_code)
    return credentials

In [99]:
# Function to unzip the dataset
def unzip_dataset(zip_file_url):
    response = requests.get(zip_file_url)
    with zipfile.ZipFile(io.BytesIO(response.content), 'r') as zip_ref:
        extracted_files = {}
        for file in zip_ref.namelist():
          if file.endswith('.csv'):
                extracted_files[file] = pd.read_csv(io.BytesIO(zip_ref.read(file)), encoding='utf-8')
    return extracted_files

In [105]:
# Unzip the dataset
zip_file_url = 'https://raw.githubusercontent.com/ariel-hedvat/AdvancedMLDLCourseAssignments/main/Experiments/Data/full_spotify_dataset_recreated.zip'
data_files = unzip_dataset(zip_file_url)

In [106]:
data_files

{'spotify_dataset_recreated.csv':                                   user_id                        artistname  \
 0        9cc0cfd4d7d7885102480dd99e7a90d6                    Elvis Costello   
 1        9cc0cfd4d7d7885102480dd99e7a90d6  Elvis Costello & The Attractions   
 2        9cc0cfd4d7d7885102480dd99e7a90d6                      Tiffany Page   
 3        9cc0cfd4d7d7885102480dd99e7a90d6  Elvis Costello & The Attractions   
 4        9cc0cfd4d7d7885102480dd99e7a90d6                    Elvis Costello   
 ...                                   ...                               ...   
 1048570  576820c2da503406e16ae9782b3c0e4c                          Dr. John   
 1048571  576820c2da503406e16ae9782b3c0e4c                    Punch Brothers   
 1048572  576820c2da503406e16ae9782b3c0e4c                    Chicago Farmer   
 1048573  576820c2da503406e16ae9782b3c0e4c               Henrik Freischlader   
 1048574  576820c2da503406e16ae9782b3c0e4c                    Curtis Harding   
 
     

In [107]:
# Function to extract track names and artist names into a dictionary
def extract_tracks_metadata(data_files):
    tracks_metadata = {}
    for df_name, df in data_files.items():
        print(f"Processing DataFrame: {df_name}")
        # Strip leading and trailing spaces from column names
        df.columns = df.columns.str.strip()
        print(f"Columns in DataFrame: {df.columns}")
        if 'trackname' in df.columns and 'artistname' in df.columns:
            print("Both 'trackname' and 'artistname' columns found.")
            for index, row in df.iterrows():
                track_name = row['trackname']
                artist_name = row['artistname']
                print(f"Extracting track: {track_name}, Artist: {artist_name}")
                if track_name not in tracks_metadata:
                    tracks_metadata[track_name] = artist_name
                else:
                    print(f"Track '{track_name}' already exists in metadata.")
        else:
            print("Either 'trackname' or 'artistname' columns missing in DataFrame.")
    return tracks_metadata


In [108]:
def get_track_metadata_from_api(tracks_metadata, sp):
    for track_name, artist_name in tracks_metadata.items():
        # Search for the track
        results = sp.search(q=f"track:{track_name} artist:{artist_name}", type='track', limit=1)
        if results['tracks']['items']:
            track_info = results['tracks']['items'][0]
            artist_uri = track_info["artists"][0]["uri"]
            artist_info = sp.artist(artist_uri)
            track_features = sp.audio_features(track_info['uri'])[0]

            # Create a list with all the metadata
            metadata_list = [
                artist_name,
                track_features['acousticness'],
                track_features['danceability'],
                track_features['energy'],
                track_features['instrumentalness'],
                track_features['speechiness'],
                track_features['liveness'],
                track_features['loudness'],
                track_features['tempo'],
                track_features['valence'],
                track_info['popularity'],
                artist_info["genres"]
            ]

            # Add the metadata list to the tracks_metadata dictionary
            tracks_metadata[track_name] = metadata_list
        else:
            print(f"No results found for track: {track_name}")


In [109]:
from google.colab import files

# Function to save tracks metadata as a CSV file
def save_tracks_metadata_to_csv(tracks_metadata, output_file):
    # Create a DataFrame from tracks_metadata dictionary
    df = pd.DataFrame.from_dict(tracks_metadata, orient='index')

    # Reset index and rename columns
    df.reset_index(inplace=True)
    df.columns = ['Track Name', 'Artist Name', 'Acousticness', 'Danceability', 'Energy', 'Instrumentalness',
                  'Speechiness', 'Liveness', 'Loudness', 'Tempo', 'Valence', 'Popularity', 'Genres']

    # Save DataFrame to CSV file
    df.to_csv(output_file, index=False)

    # Download the CSV file
    files.download(output_file)


In [ ]:
# from google.colab import files
# # Function to save tracks metadata as a CSV file
# def save_tracks_metadata_to_csv(tracks_metadata, output_file):
#     df = pd.DataFrame.from_dict(tracks_metadata, orient='index').reset_index()
#     df.columns = ['Track Name', 'Artist Name', 'Acousticness', 'Danceability', 'Energy',
#                   'Instrumentalness', 'Speechiness', 'Liveness', 'Loudness', 'Tempo',
#                   'Valence', 'Popularity', 'Genres']
#     df.to_csv(output_file, index=False)
#     # Download the CSV file
#     files.download(output_file)

In [94]:
# from google.colab import files
# # Function to save tracks metadata as a CSV file
# def save_tracks_metadata_to_csv(tracks_metadata, output_file):
#     df = pd.DataFrame.from_dict(tracks_metadata, orient='index').reset_index()
#     df.columns = ['Track Name', 'Artist Name']
#     df.to_csv(output_file, index=False)
#     # Download the CSV file
#     files.download(output_file)

In [89]:
# Extract track metadata
tracks_metadata = extract_tracks_metadata(data_files)

Streaming output truncated to the last 5000 lines.
Extracting track: Headlines - Explicit Version, Artist: Drake
Track 'Headlines - Explicit Version' already exists in metadata.
Extracting track: Heavenfallen, Artist: Goldmund
Extracting track: Heliosphan, Artist: Aphex Twin
Track 'Heliosphan' already exists in metadata.
Extracting track: Hi Scores, Artist: Boards of Canada
Track 'Hi Scores' already exists in metadata.
Extracting track: Highschool Lover, Artist: Air
Track 'Highschool Lover' already exists in metadata.
Extracting track: Homecoming, Artist: Jeff Cardoni
Track 'Homecoming' already exists in metadata.
Extracting track: Hoppípolla, Artist: Sigur Rós
Track 'Hoppípolla' already exists in metadata.
Extracting track: How We Left Fordlândia, Artist: Johann Johannsson
Track 'How We Left Fordlândia' already exists in metadata.
Extracting track: Hymn, Artist: Pete Namlook
Track 'Hymn' already exists in metadata.
Extracting track: Hyperborea, Artist: Biosphere
Extracting track: Hægt

In [90]:
tracks_metadata

{'(The Angels Wanna Wear My) Red Shoes': 'Elvis Costello',
 "(What's So Funny 'Bout) Peace, Love And Understanding": 'Elvis Costello & The Attractions',
 '7 Years Too Late': 'Tiffany Page',
 'Accidents Will Happen': 'Elvis Costello & The Attractions',
 'Alison': 'Elvis Costello',
 'All Be Okay': 'Lissie',
 'Band On The Run': 'Paul McCartney',
 'Beautiful': 'Joe Echo',
 'Blackbird - Live at CitiField, NYC - Digital Audio': 'Paul McCartney',
 'Bright Side': 'Lissie',
 'Dance The Go-Go': 'The Breakers',
 'Dance Tonight': 'Paul McCartney',
 'Decision Time': 'The Coronas',
 "Don't Dream It's Over": 'Crowded House',
 'Everywhere I Go': 'Lissie',
 'Fall At Your Feet': 'Crowded House',
 'Four Seasons In One Day': 'Crowded House',
 'Friend Like You': 'Joshua Radin',
 'Get Lost Get High Get Sick': 'The Breakers',
 'Goodtimes': 'Cocktail Slippers',
 'Helplessly Hoping': 'Crosby, Stills & Nash',
 'Here Before': 'Lissie',
 'Here We Go': 'Cocktail Slippers',
 'I Am The Blaze': 'Tiffany Page',
 "I Do

In [110]:
# Read Spotify credentials from URL
spotify_credentials_url = 'https://raw.githubusercontent.com/ariel-hedvat/AdvancedMLDLCourseAssignments/main/Experiments/Data/spotify_credentials.txt'
spotify_credentials = read_spotify_credentials_from_url(spotify_credentials_url)

# Initialize Spotipy client with the obtained credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=spotify_credentials['CLIENT_ID'],
                                                           client_secret=spotify_credentials['CLIENT_SECRET']))

In [111]:
# Assuming tracks_metadata is already populated with track names and artist names
# Call the function to retrieve metadata for each track
get_track_metadata_from_api(tracks_metadata, sp)

No results found for track: 7 Years Too Late
No results found for track: Blackbird - Live at CitiField, NYC - Digital Audio
No results found for track: Bright Side
No results found for track: I'd Rather Be With You [Radio Edit]
No results found for track: Oliver's Army
No results found for track: Everybody's On the Run
No results found for track: Player One - Feat. Ddash Of Ddamage
No results found for track: 0x10c - 01
No results found for track: 0x10c - 02
No results found for track: Afterglow
No results found for track: Can't Keep Giving In - Original Mix
No results found for track: Exile Vilify (From the Game Portal 2)
No results found for track: Hey Now - J'adore Dior Remix by The Shoes
No results found for track: Innerverse
No results found for track: It's Raining Clouds
No results found for track: Magician's Assistant
No results found for track: Pio - Original Mix


KeyboardInterrupt: 

In [ ]:
tracks_metadata

In [ ]:
# Read Spotify credentials from URL
spotify_credentials_url = 'https://raw.githubusercontent.com/ariel-hedvat/AdvancedMLDLCourseAssignments/main/Experiments/Data/spotify_credentials.txt'
spotify_credentials = read_spotify_credentials_from_url(spotify_credentials_url)
CLIENT_ID = spotify_credentials['CLIENT_ID']
CLIENT_SECRET = spotify_credentials['CLIENT_SECRET']

# Initialize Spotipy client
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET))

In [95]:
# Save tracks metadata to CSV
output_file = 'tracks_metadata.csv'
save_tracks_metadata_to_csv(tracks_metadata, output_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

old: